In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

# Manipulación de datos con [*pandas*](https://pandas.pydata.org/)

En este clase veremos 
- como crear dataframes a partir de datos de distintas fuentes 
- funciones avanzadas de manipulación de dataframes

In [ ]:
import pandas as pd
print("Versión de pandas "+pd.__version__)
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display

## Lectura de archivos CSV (Comma-Separated Values)

Un archivo CSV es una tabla en formato texto plano cuyas columnas están separadas por comas

Descarguemos la base de datos "Dow Jones Index" del repositorio UCI

https://archive.ics.uci.edu/ml/datasets/Dow+Jones+Index

In [ ]:
%%bash
wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00312/dow_jones_index.zip
unzip -o dow_jones_index.zip

Visualizemos el archivo csv

In [ ]:
!head dow_jones_index.data

Pandas provee la función [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) para importar tablas en formato texto plano

In [ ]:
df = pd.read_csv("dow_jones_index.data", sep=',', header=0, index_col='stock')
df.head()

Algunas columnas se han guardado como strings

In [ ]:
df.dtypes

In [ ]:
display(df["date"][0],
        df["open"][0])

## Parsing: Selección manual de formato de dato

Hasta ahora hemos dejado que pandas asigne los tipos de dato de manera automática

La función `read_csv` tiene un argumento `converters` que recibe un diccionar de funciones

Esto puede usarse para *parsear* manualmente las columnas que no se importaron automaticamente como deseabamos

In [ ]:
conv = dict.fromkeys(['open', 'close', 'high', 'low', 
                      'next_weeks_open', 'next_weeks_close'], lambda x: float(x.strip("$")))

Pandas fue diseñado para analizar series de tiempo e incorpora la función [`to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) que toma un string y retorna un `Timestamp`

Por defecto el formato se infiere, pero puede forzarce usando el argumento `format`

In [ ]:
display(pd.to_datetime("1/5/2018"),
        pd.to_datetime("1st of May of 2018"),
        pd.to_datetime("May/1/2018"),
        pd.to_datetime("2018"),
        pd.to_datetime("14:45"),
        pd.to_datetime("May/1/2018 14:45"))

Para facilitarnos la vida `read_csv` tiene un argumento `parse_dates` que recibe una lista de enteros especificando las columnas que queremos convertir a fechas

In [ ]:
df = pd.read_csv("dow_jones_index.data", sep=',', header=0, index_col='stock', 
                 converters=conv, parse_dates=[2])
df.head()

Los tipos de la nueva tabla son:

In [ ]:
df.dtypes

Ahora que los tiempos tiene formato *timestamp* podemos usarlos como índice

Esto nos permite recuperar rapidamente todos los eventos dentro de un intervalo de tiempo

In [ ]:
df = pd.read_csv("dow_jones_index.data", sep=',', header=0, index_col='date', 
                 converters=conv, parse_dates=[2])

df[df["stock"] == "AA"].loc["2011-02-01":"2011-03-12"]

Matplotlib tiene funciones para parsear datos temporales

In [ ]:
import matplotlib.dates as md

fig, ax = plt.subplots(figsize=(7, 4))
sub_df = df[df["stock"] == "AA"]
for x, o, c in zip(sub_df.index.values, sub_df['open'].values, sub_df['close'].values):
    ax.arrow(x=md.date2num(x), y=o, dx=0, dy=c-o, head_width=3, head_length=0.1, fc='k', ec='k')
ax.fill_between(sub_df.index.values, sub_df['low'].values, sub_df['high'].values, alpha=0.5);

## Lectura de archivos excel

- Pandas provee la función [`read_excel`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html)

- Requisito adicional: [python-xlrd](https://github.com/python-excel/xlrd)

In [ ]:
!wget -c http://www.censo2017.cl/wp-content/uploads/2017/12/Cantidad-de-Viviendas-por-Tipo.xlsx

In [ ]:
df = pd.read_excel("Cantidad-de-Viviendas-por-Tipo.xlsx", sheet_name=1, 
                   usecols=list(range(1, 20)), header=1, index_col='ORDEN')

df.dropna(inplace=True)
df.drop(0, inplace=True)
display(df.head())
df.describe()

Podriamos querer obtener los valores totales de la Provincia de Valdivia: **reducción suma**

In [ ]:
col_mask = df.columns[4:-1]
display(col_mask)
row_mask = df["NOMBRE PROVINCIA"] == "VALDIVIA"
display(df.loc[row_mask].head())
df.loc[row_mask, col_mask].sum()

## Cambio de índice

Podemos usar las funciones [reset_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html) y [set_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html#pandas.DataFrame.set_index) para modificar el índice del dataframe a nuestra conveniencia

In [ ]:
df = df.reset_index()
df = df.set_index("NOMBRE PROVINCIA")
display(df.head())
df.loc["VALDIVIA", col_mask].sum()

In [ ]:
df = df.reset_index()
df = df.set_index("ORDEN")
%timeit -n20 df.loc[df["NOMBRE PROVINCIA"] == "VALDIVIA", col_mask].sum()
df = df.reset_index()
df = df.set_index("NOMBRE PROVINCIA")
%timeit -n20 df.loc["VALDIVIA", col_mask].sum()
df = df.reset_index()
df = df.set_index("ORDEN")

## Groupby: Reducciones condicionales

En el caso anterior podemos reducir de forma separada para cada región o provincia sin cambiar índices

La función [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) permite hacer una reducción condicional a una etiqueta

Podemos imaginar que la función `groupby` es una [secuencia](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) como la siguiente:


Donde
- *Split*: divide los datos según una **llave**
- *Apply*: Realiza una función sobre cada grupo: reducción, transformación, filtrado
- *Combine*: Mezcla el resultado en un nuevo dataframe donde la **llave** se convierte en el índice

In [ ]:
df_redu = df.groupby("NOMBRE REGIÓN").sum()
display(df_redu.head())
fig, ax = plt.subplots(figsize=(6, 7), tight_layout=True)
df_redu.plot(ax=ax, y=0, kind='bar', logy=True);

`groupby` puede usarse como iterador

In [ ]:
for (region, sub_df) in df.groupby('NOMBRE REGIÓN'):
    display(region, sub_df.head(1))

Notar que no estamos limitados a las reducciones definidas

Podemos usar los atributos de `groupby` para obtener más funcionalidad
- `aggregate` : Operación de reducción
- `filter` : Operación de eliminación de filas (drop)
- `transform` : Operación de modificación columna a columna
- `apply`: Aplica una función arbitraria 

In [ ]:
# Las funciones de reducción deben recibir un arreglo y retornar un valor
def mi_reduccion(x):
    return np.sum(x**np.log10(x+1e-10))

# Argumento lista de funciones: Cada función se aplica a todas las columnas
display(df.groupby("NOMBRE REGIÓN").aggregate([np.median, mi_reduccion]).head())
# Argumento diccionario de funciones: una función distinta por columna
display(df.groupby("NOMBRE REGIÓN").aggregate({'Viviendas Particulares Ocupadas con Moradores Presentes': np.median, 
                                               'Viviendas Particulares Ocupadas con Moradores Ausentes': mi_reduccion}).head())

In [ ]:
def mi_filtro(x):
    return x["Viviendas Particulares Ocupadas con Moradores Presentes"].sum() > 500000

# El filtro debe establecer una condición sobre el grupo completo
# En este caso retorna df menos las comunas de las regiones con menos de 500.000 viviendas

df.groupby("NOMBRE REGIÓN").filter(mi_filtro)

In [ ]:
def mi_transformación(x):
    if x.dtype == np.float:
        return (x - x.mean())/x.std()
    else:
        return x 

# La transformación opera columna por columna
# La transformación debe retornar un resultado que es del mismo tamaño de la entrada

df.groupby("NOMBRE REGIÓN").transform(mi_transformación)

In [ ]:
def mi_funcion(x):
    col = 'Viviendas Particulares Ocupadas con Moradores Presentes'
    x[col] -= x[col].mean()
    return x


df.groupby("NOMBRE REGIÓN").apply(mi_funcion)

## Splitting

Podemos usar más de una etiqueta para hacer el splitting de `groupby`

Podemos también usar una función, una lista, diccionario o dataframe

Refierase a la [documentación](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) para más detalles

In [ ]:
df.groupby(["NOMBRE REGIÓN", "NOMBRE PROVINCIA"]).sum()

## Multi índices

Hemos notado que la tabla tiene estructura jerárquica: REGION, PROVINCIA, COMUNA

Podemos usar un multi-índice en pandas para manipular mejor esta tabla

Podemos crear un multi-índice usando la modulo `MultiIndex` y sus funciones `from_array`, `from_frame` y `from_tuple`

Luego podemos aplicarlo con la función `set_index`

In [ ]:
# Set_index acepta una lista con nombres de columna:
df.reset_index()
df = df.set_index(["NOMBRE REGIÓN", "NOMBRE PROVINCIA"])
df.head(10)

In [ ]:
display(df.index)

Indexando y haciendo slicing con dataframes multi-indexados

Usamos una tupla para especificar los índices primario y secundario

Se usa el objeto [`IndexSlice`](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.IndexSlice.html) para generar slices

In [ ]:
idx = pd.IndexSlice

display(df.loc["LOS RÍOS"],
        df.loc[("LOS LAGOS", "OSORNO")],
        df.loc[idx[:, "VALDIVIA"], :],
        df.loc[("LOS RÍOS", "RANCO"), "Viviendas Particulares Ocupadas con Moradores Presentes":])

## La función `eval` y la función `query` 

[`eval`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.eval.html) y [`query`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) son atributos de DataFrame que permiten evaluar una expresión arbitraria o hacer consultas (filtro) sobre las columnas del dataframe, respectivamente

Están basadas en [`numexpr`](https://github.com/pydata/numexpr) que es un evaluador de expresiones numéricas acelerado para ndarray (rendimiento casi C)

`numexpr` acepta un string con una expresión estilo numpy, la evalua y returna el resultado

Si los arreglos son grandes ganamos en velocidad y en memoria

In [ ]:
import numexpr as ne

A = np.arange(100000).reshape(1000, 100)

# NumPy es más lento ya que evalua y guarda cada paso
%timeit -n10 np.tanh(-A**2) > np.exp(np.cos(A)/2)
%timeit -n10 ne.evaluate("tanh(-A**2) > exp(cos(A)/2)")

b1 = np.tanh(-A**2) > np.exp(np.cos(A)/2)
b2 = ne.evaluate("tanh(-A**2) > exp(cos(A)/2)")
np.allclose(b1, b2)

**OJO:** Nuestras columnas no pueden tener espacios en sus nombres para poder usar `query`/`eval`

### Renombrando las columnas

Podemos 

- Usar el atributo `rename` y especificar los nuevos nombres uno a uno 
- Aplicar operaciones de string al atributo `columns`

In [ ]:
df = pd.read_excel("Cantidad-de-Viviendas-por-Tipo.xlsx", sheet_name=1, 
                   usecols=list(range(1, 20)), header=1, index_col='ORDEN')
df.dropna(inplace=True)
df.drop(0, inplace=True)
# df.rename(columns={'NOMBRE PROVINCIA': 'NOMBRE_PROVINCIA'}, inplace=True)
df.columns = df.columns.str.replace(' ', '_')

### Uso de eval y query

Hacemos operaciones o consultas sobre las columnas usando su etiqueta

Para `query` podemos juntar varias consultas con `and` y `or`

Podemos llamar variables externas anteponiendo un `@`

Al igual que con numexpr las operaciones intermedias no se guardan en memoria

- Opinión objetiva: Si el dataframe es grande ganamos en velocidad en uso de memoria
- Opinion subjetiva: En general ganamos en legibilidad

Forma tradicional versus `eval`

In [ ]:
a = df['Viviendas_Particulares_Ocupadas_con_Moradores_Presentes']/df['TOTAL_VIVIENDAS']
b = df.eval('Viviendas_Particulares_Ocupadas_con_Moradores_Presentes/TOTAL_VIVIENDAS')
np.allclose(a, b)
display(b.head())

Podemos evaluar y guardar el resultado directamente en el dataframe

In [ ]:
df.eval('Proporcion_encuestas_vs_total = Viviendas_Particulares_Ocupadas_con_Moradores_Presentes/TOTAL_VIVIENDAS', inplace=True)
df.head()

Filtrando con `query`

In [ ]:
percentage = 0.75
df.query('NOMBRE_PROVINCIA == "VALDIVIA" \
and Viviendas_Particulares_Ocupadas_con_Moradores_Presentes/TOTAL_VIVIENDAS > @percentage').head()

## Lectura de bases de datos SQL

Pandas puede usarse para leer y hacer consultas a una base de datos en formato SQL

Primero vamos a crear una base de datos e insertar un dataframe como tabla

Usaremos [sqlite3](https://docs.python.org/3/library/sqlite3.html) que es parte de la librería estándar de Python

- sqlite permite conectar a una base de datos local: RAM, disco, o disco externo montado
- sqlite no está diseñado para soportar múltiples usuarios conectados a una misma base de datos
- Alternativas: [SQL Alchemy](https://www.sqlalchemy.org/), [PostgreSQL+Python](http://initd.org/psycopg/), [Peewee](http://docs.peewee-orm.com/en/latest/)


In [ ]:
import sqlite3 
# Abrimos una conexión
conn = sqlite3.connect(":memory:")
# La palabra clave :memory: corresponde a una base de datos en memoria RAM
df.to_sql("censo_viviendas", conn, if_exists='replace', index=False)

Podemos usar [`read_sql_query()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) para hacer una consulta y retornar un dataframe

Por ejemplo si queremos toda la tabla:

In [ ]:
sql_string = "SELECT * FROM censo_viviendas limit 10"
print(sql_string)
pd.read_sql_query(sql_string, conn)

O si quieremos un subconjunto específico:

In [ ]:
sql_string = "SELECT [{0}], [{1}] FROM censo_viviendas WHERE [{2}] = 'VALDIVIA' limit 5".format("Viviendas_Particulares_Ocupadas_con_Moradores_Presentes", 
                                                                                                "NOMBRE_COMUNA",
                                                                                                "NOMBRE_PROVINCIA")
print(sql_string)
pd.read_sql_query(sql_string, conn)

In [ ]:
# Cerramos la conexión a la base de datos
conn.close()

## Guardar y leer una tabla en formato JSON

Podemos usar el atributo [`to_json`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html) para convertir un dataframe a este formato

El keyword `orient` nos permite seleccionar como organizar el scheme del json

Las opciones son `columns` (por defecto), `table`, `values`, `index`, `split` y `records`

In [ ]:
df.to_json("pandas.json", orient='table')

!head -c 200 pandas.json
print("")
!ls -lah pandas.json

In [ ]:
# Tenemos que usar la misma orientación con la que lo guardamos
pd.read_json("pandas.json", orient='table').head()

## Guardar y leer una tabla en formato HDF5

Podemos usar el atributo `to_hdf` para convertir nuestra tabla a formato HDF5

Para acceso de más bajo nivel podemos usar la clase `HDFStore`

Para lectura podemos usar la función `read_hdf`

NO FUNCIONA CON NUMPY MAS RECIENTE: https://github.com/PyTables/PyTables/issues/719

In [ ]:
df.to_hdf("pandas_hdf.h5", key='asd', mode='w')

mi_tabla_recuperada = pd.read_hdf("pandas_hdf.h5", key='/asd', mode='r')